# Criando um Web App de Análise de Sentimento
## Usando PyTorch e SageMaker

_Esse tutorial foi feito com base no projeto final do Nanodegree em Deep Learning da Udacity_

---

Agora que vocês já conhecem um pouco de AWS e de Sagemaker, iremos construir um projeto completo (end to end) com o intuito de desenvolver um web app onde o usuário insira um texto de avaliação de filme e saiba se essa avaliação é positiva ou negativa.

## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

For this project, you will be following the steps in the general outline with some modifications. 

First, you will not be testing the model in its own step. You will still be testing the model, however, you will do it by deploying your model and then using the deployed model by sending the test data to it. One of the reasons for doing this is so that you can make sure that your deployed model is working correctly before moving forward.

In addition, you will deploy and use your trained model a second time. In the second iteration you will customize the way that your trained model is deployed by including some of your own code. In addition, your newly deployed model will be used in the sentiment analysis web app.

In [1]:
# Make sure that we use SageMaker 1.x
!pip install sagemaker==1.72.0

     |████████████████████████████████| 297 kB 16.0 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=386358 sha256=e148b841145c44ca84a4b4c0559d270b36709a512f06072e33dd7eacb98666bb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.41.0
    Uninstalling sagemaker-2.41.0:
      Successfully uninstalled sagemaker-2.41.0


## Passo 1: Baixando os dados

Para o nosso projeto, usaremos uma base bastante conhecida chamada [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [2]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2021-05-31 23:46:19--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  8.31MB/s    in 13s     

2021-05-31 23:46:33 (6.09 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Passo 2: Preparando e processando os dados

Processaremos os nossos dados a fim de deixá-los em um formato mais fácil para realizar o treinamento do modelo. Primeiramente, iremos unir as avaliação positivas e negativas em uma mesma estrutura de dados, uma vez que os dados vem em arquivos separados. Após isso iremos separá-los em treino e teste, garantindo que eles estejam misturados.

In [3]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [4]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Agora, vamos dar uma olhadinha nos nossos dados!

In [7]:
print(train_X[100])
print(train_y[100])

Everyone else who has commented negatively about this film have done excellent analysis as to why this film is so bloody awful. I wasn't going to comment, but the film just bugs me so much, and the writer/director in particular. So I must toss in my hat to join the naysayers.<br /><br />I saw the original "Wicker Man" and really loved the cornucopia of music, sensuality, paganism in a modern world, and the clash of theological beliefs. This said, I am not part of the crowd that thinks remakes of great movies shouldn't be done. For example, I liked the original 1950's "Invasion of the Body Snatchers", but equally enjoyed the 1978 remake. Both films can stand on their own. Another example is "The Thing". The original, as campy as it looks compared to today's standards, has a lot to be proud of in the 1982 remake with Kurt Russell (my all time favorite horror movie). So that small minority of people who like "The Wicker Man" re-make can not accuse me of dissing this piece of crap just bec

Como primeiro passo de pré processamento, iremos limpar as tags HTML que podem aparecer nas avaliações e, após isso, iremos _tokenizar_ nossos dados para que palavras como *entertained* e *entertaining* sejam consideradas iguais no nosso modelo.

In [71]:
import nltk
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

A função `review_to_words` utiliza a biblioteca `BeautifulSoup` para remover as tags HTML e usa a biblioteca `nltk` para tokenizar as avaliações e remover as stopwords.  
Abaixo, podemos ver o output dessa função aplicado a uma avaliação.

In [9]:
review_to_words(train_X[100])

['everyon',
 'els',
 'comment',
 'neg',
 'film',
 'done',
 'excel',
 'analysi',
 'film',
 'bloodi',
 'aw',
 'go',
 'comment',
 'film',
 'bug',
 'much',
 'writer',
 'director',
 'particular',
 'must',
 'toss',
 'hat',
 'join',
 'naysay',
 'saw',
 'origin',
 'wicker',
 'man',
 'realli',
 'love',
 'cornucopia',
 'music',
 'sensual',
 'pagan',
 'modern',
 'world',
 'clash',
 'theolog',
 'belief',
 'said',
 'part',
 'crowd',
 'think',
 'remak',
 'great',
 'movi',
 'done',
 'exampl',
 'like',
 'origin',
 '1950',
 'invas',
 'bodi',
 'snatcher',
 'equal',
 'enjoy',
 '1978',
 'remak',
 'film',
 'stand',
 'anoth',
 'exampl',
 'thing',
 'origin',
 'campi',
 'look',
 'compar',
 'today',
 'standard',
 'lot',
 'proud',
 '1982',
 'remak',
 'kurt',
 'russel',
 'time',
 'favorit',
 'horror',
 'movi',
 'small',
 'minor',
 'peopl',
 'like',
 'wicker',
 'man',
 'make',
 'accus',
 'diss',
 'piec',
 'crap',
 'make',
 'film',
 'solidifi',
 'neil',
 'labut',
 'sexism',
 'misogynist',
 'tendenc',
 'also',
 'ma

Já a função abaixo, `preprocess_data`, aplica a função `review_to_words` para cada uma das avaliações dos datasets de treino e teste. Além disso, ela também faz o cache dos dados, para que, caso algo aconteça, você possa voltar o pré processamento de onde parou.

In [10]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [11]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocessed_data.pkl


### Transformando os dados

Agora, nós iremos construir uma representação dos nossos dados muito similar a representação conhecida como bag-of-words. Para a rede neural recorrente que iremos usar, nós iremos relizar a transformação dos dados da seguinte forma:

1. Transformar cada palavra em um número inteiro;
2. Definir um tamanho para nosso vocabulário, ou seja, iremos remover palavras que aparecem pouco (para essas palavras atribuiremos o mesmo número inteiro (1));
3. Como estamos usando uma RNN, defineros um tamanho para nossas sequência, ou seja, truncaremos aquelas que forem maiores e iremos inserir um caractér (0) para quando a avaliação for menor do que o tamanho definido.

Começaremos construindo uma função que nos retorna um dicionário de tamanho especifíco e com as palavras que mais aparecem. Não podemos esquecer de reservar o índice 0 e 1 para os caractéres vazio e pouco frequente!

In [29]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for review in data:
        for word in review:
            if word_count.get(word) is None:
                word_count[word] = 1
            else:
                word_count[word] += 1
    
    sorted_words = [k for k,v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)]
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [30]:
word_dict = build_dict(train_X)

### Salvando nosso dicionário

Mais para frente, quando tivermos nosso modelo, teremos que usar nosso dicionário para realizar as predições. Sendo assim, precisamos salvá-lo para usar no futuro!

In [32]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [33]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transformando as avaliações

Agora, é hora de convertermos nossas avaliações de treino e teste para a sequência de número inteiros de tamanho fixo que entrará na nossa rede neural!

In [34]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [35]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

Apenas para validar se nossa função está fazendo tudo corretamente, vamos verificar um dado da nossa base de treino e ver seu tamanho.

In [36]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(train_X[100])
print(len(train_X[100]))

[ 226  265  329 1287    3  143  222 3939    3 1559  299   25  329    3
 1694   21  308   67  765  130 2932 1735 1352    1  135   90    1   55
   16   29    1   85 4638    1  583   92 4147    1 1713  233   63 1611
   30  822   26    2  143  360    5   90 1483 3514  458    1  866   77
 4322  822    3  390   79  360   35   90 2253   19  509  433  695   70
 2320    1  822 2734 1955    6  395  102    2  340 1079   23    5    1
   55    8 2166    1  267  522    8    3    1 2928    1    1    1 4291
   27   34  108  826   50    8  566  529   15 1134  359  217 1104  544
   46   18 1491  382  305 1479    1 1170  240   89   15 2547    5  915
  326 1030    1   90  331  795  283  305  866 2221    1  341 1808 2752
  216    9  333  226  774  544  556    1 2843    1 4974  453  124 1052
  228 2981 4974  130  262  187 1491    9    1    1  749  108  283  915
  257   64   65 1001 1014  368  305  775    5    1  305  221 2135  271
  252   39 2418  252   72 1701  302   89 1383 1318    1  283   16   50
  640 

## Passo 3: Subindo nossos dados de treino para o S3

Nós iremos precisar subir nossos dados de treino para o S3 para que possamos acessá-lo durante o treinamento.

### Salvando os dados de treino localmente

Antes de subir para o S3, iremos salvar nossos dados localmente. É muito importante saber a estrutura dos dados que vamos salvar, para que possamos utilizar de forma correta. No nosso caso, as linhas do nosso dataset irão ter a forma (colunas): `label`, `length`, `review[500]`, onde `review[500]`é a sequência com 500 números inteiros que geramos acima.

In [37]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Subindo para o S3

In [38]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [39]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**OBS:** A célula acima sobe todos os arquivos contidos no nosso diretório para o S3. Isso inclui o `word_dict.pkl`, que usaremos na hora de realizar uma nova predição e garantir que o pré processamento dos novos dados seja o mesmo dos dados de treino. 

## Passo 4: Construindo e treinando nosso modelo em Pytorch

In the XGBoost notebook we discussed what a model is in the SageMaker framework. In particular, a model comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. In the XGBoost example we used training and inference code that was provided by Amazon. Here we will still be using containers provided by Amazon with the added benefit of being able to include our own custom code.

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [40]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

The important takeaway from the implementation provided is that there are three parameters that we may wish to tweak to improve the performance of our model. These are the embedding dimension, the hidden dimension and the size of the vocabulary. We will likely want to make these parameters configurable in the training script so that if we wish to modify them we do not need to modify the script itself. We will see how to do this later on. To start we will write some of the training code in the notebook so that we can more easily diagnose any issues that arise.

First we will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as we do not have access to a gpu and the compute instance that we are using is not particularly powerful. However, we can work on a small bit of the data to get a feel for how our training script is behaving.

In [41]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### (TODO) Writing the training method

Next we need to write the training code itself. This should be very similar to training methods that you have written before to train PyTorch models. We will leave any difficult aspects such as model saving / loading and parameter loading until a little later.

In [42]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            output = model.forward(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [43]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6934759020805359
Epoch: 2, BCELoss: 0.6848492383956909
Epoch: 3, BCELoss: 0.6776467561721802
Epoch: 4, BCELoss: 0.6698715329170227
Epoch: 5, BCELoss: 0.6606590151786804


In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### (TODO) Training the model

When a PyTorch model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained. Inside of the `train` directory is a file called `train.py` which has been provided and which contains most of the necessary code to train our model. The only thing that is missing is the implementation of the `train()` method which you wrote earlier in this notebook.

**TODO**: Copy the `train()` method written above and paste it into the `train/train.py` file where required.

The way that SageMaker passes hyperparameters to the training script is by way of arguments. These arguments can then be parsed and used in the training script. To see how this is done take a look at the provided `train/train.py` file.

In [46]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [47]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-06-01 00:57:13 Starting - Starting the training job...
2021-06-01 00:57:14 Starting - Launching requested ML instances......
2021-06-01 00:58:24 Starting - Preparing the instances for training......
2021-06-01 00:59:29 Downloading - Downloading input data...
2021-06-01 00:59:54 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-06-01 01:00:16,473 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-06-01 01:00:16,476 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-06-01 01:00:16,487 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-06-01 01:00:16,491 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-06-01 01:00:16,747 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-06-01 01:00:16,7

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Now that we have trained our model, we would like to test it to see how it performs. Currently our model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`. Fortunately for us, SageMaker provides built-in inference code for models with simple inputs such as this.

There is one thing that we need to provide, however, and that is a function which loads the saved model. This function must be called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file which we specified as the entry point. In our case the model loading function has been provided and so no changes need to be made.

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.

**NOTE:** When deploying a model you are asking SageMaker to launch an compute instance that will wait for data to be sent to it. As a result, this compute instance will continue to run until *you* shut it down. This is important to know since the cost of a deployed endpoint depends on how long it has been running for.

In other words **If you are no longer using a deployed endpoint, shut it down!**

**TODO:** Deploy the trained model.

In [48]:
# TODO: Deploy the trained model
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [49]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [50]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [51]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.85348

**Question:** How does this model compare to the XGBoost model you created earlier? Why might these two models perform differently on this dataset? Which do *you* think is better for sentiment analysis?

**Answer:**

Acc XGBoost: 0.86116

Acc RNN: 0.85348

As we can see, the two models achieve very close accuracy, but we train only for 10 epochs our RNN and, maybe, if we train for more epochs we can achieve higher accuracy than in the XGBoost. I think that RNN will perform better due to the nature of the problem, which is a liguistic problem, so, the order of the words in a sentece have a huge impact in the final prediction.

### (TODO) More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [53]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order process the review we will need to repeat these two steps.

**TODO**: Using the `review_to_words` and `convert_and_pad` methods from section one, convert `test_review` into a numpy array `test_data` suitable to send to our model. Remember that our model expects input of the form `review_length, review[500]`.

In [58]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data, len_test_data = convert_and_pad(word_dict, review_to_words(test_review))
test_data = np.array(test_data)

Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [59]:
predictor.predict([test_data])

array(0.6170426, dtype=float32)

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Of course, just like in the XGBoost notebook, once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [62]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

As we saw above, by default the estimator which we created, when deployed, will use the entry script and directory which we provided when creating the model. However, since we now wish to accept a string as input and our model expects a processed review, we need to write some custom inference code.

We will store the code that we write in the `serve` directory. Provided in this directory is the `model.py` file that we used to construct our model, a `utils.py` file which contains the `review_to_words` and `convert_and_pad` pre-processing functions which we used during the initial data processing, and `predict.py`, the file which will contain our custom inference code. Note also that `requirements.txt` is present which will tell SageMaker what Python libraries are required by our custom inference code.

When deploying a PyTorch model in SageMaker, you are expected to provide four functions which the SageMaker inference container will use.
 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.

For the simple website that we are constructing during this project, the `input_fn` and `output_fn` methods are relatively straightforward. We only require being able to accept a string as input and we expect to return a single value as output. You might imagine though that in a more complex application the input or output may be image data or some other binary data which would require some effort to serialize.

### (TODO) Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [60]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

As mentioned earlier, the `model_fn` method is the same as the one provided in the training code and the `input_fn` and `output_fn` methods are very simple and your task will be to complete the `predict_fn` method. Make sure that you save the completed file as `predict.py` in the `serve` directory.

**TODO**: Complete the `predict_fn()` method in the `serve/predict.py` file.

### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [86]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [87]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [88]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.84

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [90]:
predictor.predict(test_review)

b'1.0'

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 8: Criando nossa função Lambda

Construimos nosso endpoint e deployamos nosso modelo, mas... e se quisessemos que nosso modelo fosse acessável por meio de um webapp? Para isso, precisamos construir alguns outros componentes, que podemos ver na arquitetura abaixo:

<img src="Web App Diagram.svg">

Indo da esquerda para direita temos: 

- A EC2 será responsável por subir nossa aplicação Flask, servir nossa página e enviar a requisição para o API Gateway;
- O API Gateway receberá a requisição e encaminhará para a função Lambda;
- Já a função Lambda funcionará como redirecionando o texto da nova avaliação para o Sagemaker endpoint, uma vez que não há como realizar a integração API Gateway -> Sagemaker Endpoint de forma nativa e também não podemos chamar a URL do Sagemaker Endpoint sem estar dentro da AWS.

### Configurando nossa função Lambda

A primeira coisa que faremos será a criação da nossa função Lambda. Nossa função receberá como input os dados vindos do nosso API Gateway, irá realizar a chamada do endpoint e retornará a resposta para o API Gateway.

#### Parte A: Criando IAM Role para a Lambda

Já que queremos que nossa função Lambda chame o Sagemaker Endpoint, precisamos garantir que ele tenha permissão para isso. Dessa forma, precisamos adicionar essa permissão dentro da Role que usaremos em nossa função Lambda.

Usando o Console da AWS, procuraremos por **IAM** na barra de busca e clicaremos em **Roles** no menu esquerdo. Feito isso, clique em **Create Role**. Garante que em **AWS service** o time de _trusted entity_ selecionado seja **Lambda** e, em seguida, clique em **Next: Permissions**.

Na barra de busca procure por `sagemaker` e clique no checkbox referente a **AmazonSageMakerFullAccess** policy. Clique em **Next: Review**.

Por último, dê um nome para usa role e garanta que você irá se lembrar na hora de criar sua função Lambda!

Usaremos o nome `LambdaSageMakerRole`.

#### Parte B: Criando a Lambda

Agora, é hora de criarmos nossa função Lambda!

Para isso, no console AWS, procure por Lambda e clique em **Create a function**. Na página seguinte, clique em **Author from scratch**, selecione o runtime como sendo Python, dê um nome para sua função, como por exemplo: `sentiment_analysis_func`. Não esqueça de selecionar a role que criamos!

Após isso, clique em **Create Function**.

Na próxima página você verá algumas informações sobre sua função Lambda que você acabou de criar. Se você descer um pouco a página, verá um editor de texto onde você pode escrever código que será executado quando sua função for chamada. No nosso projeto, usaremos o código abaixo (basta copiar e colar) e não esqueça de mudar o `EndpointName` para o nome do endpoint que pegaremos na célula abaixo.

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

In [91]:
predictor.endpoint

'sagemaker-pytorch-2021-06-01-03-42-13-849'

Após alterar o nome do endpoint na sua função Lambda, clique em **Salvar**. Feito, sua função Lambda estará pronto para ser executada!

### Criando e configurando o API Gateway

Agora, é hora de criarmos nosso API Gateway!

Abra o console da AWS e busque por **API Gateway** e, na página inicial, procure por REST API e clique em **Build**. Na tela de configuração, em **Create new API** selecione **New API** e defina um nome para seu Gateway e clique em **Create API**.

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**. 

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_api`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 8: Deployando nosso web app

Agora, como passo final, faremos o deploy do nosso web app, para que possamos acessá-lo pelo browser e saber o sentimento de novas avaliações de filmes!  
Para isso, usaremos uma instância de EC2 e subiremos uma aplicação Flask que será responsável por carregar a página e fazer as chamadas no API Gateway.

Assim, primeiramente, é necessário criar criar uma instância de EC2 e configurar seu `security group` para garantir que seja permitido o ingresso pela porta 80 (Adicionar uma inbound rule permitindo HTTP para qualquer Ipv4).

Após isso, vamos acessar, via browser, o terminal da máquina que criamos e instalar o git (```yum install git```). Com o git instalado, iremos executar o comando ```git clone https://github.com/pedrogengo/aws-deploy-example.git```.

Feito isso, iremos acessar a pasta `website\templates` e abrir o arquivo chamado `index.html` (```vim index.html```) para que possamos alterar a URL da nossa API. Procure por uma linha que contém **\*\*REPLACE WITH PUBLIC API URL\*\*** e insira a url da sua API onde será possível realizar a predição. Feito isso, salve o arquivo (```Esc -> :wq```) e volte para o diretório `website` (```cd ..```)`.

Já no repositório `website`, execute o comando ```pip3 install -r requirements.txt``` e, depois disso, o comando ```python3 -m flask run --host 0.0.0.0 --port 80```.

Pronto! Basta acessar o Ip da sua máquina e ver sua aplicação funcionando!

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [92]:
predictor.delete_endpoint()